# Create Directories For Cellranger Outputs

In [4]:
import pandas as pd
import subprocess
import os
import shutil

In [2]:
res_df = pd.read_csv('/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/Tsai/Preprocessing/FASTQ_Transfer/CSVs/SocIsl_Tsai_To_Openmind.csv')
res_df.head()

,Unnamed: 0,Batch,projid,Library_ID,Tsai_path,Openmind_Destination,Backup
0,0,last_16,50300822,D19-10914_3,/nfs/picower001/lhtsailab/lab_shared/lpantano/...,/om/scratch/Mon/mabdel03/SocIsl/FASTQs/5030082...,NaN
1,1,last_16,51624179,D19-10914_8,/nfs/picower001/lhtsailab/lab_shared/lpantano/...,/om/scratch/Mon/mabdel03/SocIsl/FASTQs/5162417...,NaN
2,2,190501Tsa_SM-IXFTU,50302266,D19-4106,/nfs/picower001/lhtsailab/lab_shared/lpantano/...,/om/scratch/Mon/mabdel03/SocIsl/FASTQs/5030226...,NaN
3,3,190501Tsa_SM-IXFTU,50302266,D19-4106,/nfs/picower001/lhtsailab/lab_shared/lpantano/...,/om/scratch/Mon/mabdel03/SocIsl/FASTQs/5030226...,NaN
4,4,190501Tsa_SM-IXFTU,46251007,D19-4115,/nfs/picower001/lhtsailab/lab_shared/lpantano/...,/om/scratch/Mon/mabdel03/SocIsl/FASTQs/4625100...,NaN


In [6]:
projids = set(res_df['projid'])
root = '/om/scratch/Mon/mabdel03/SocIsl/Counts'


empty_projids = set()

for projid in projids:
    folder_path = os.path.join(root, str(projid))
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        if not os.listdir(folder_path):
            empty_projids.add(projid)
        else:
            pass
    else:
        empty_projids.add(projid)

# Generate Cellranger Batch Scripts

In [8]:
parent_scripts = '/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/Tsai/Preprocessing/Counts/SocIsl/Batch_Scripts'
for num in empty_projids:
    res_sub = res_df[res_df['projid'] == num]
    res_sub = res_sub.reset_index(drop=True)

    libID = res_sub['Library_ID'][0]
    fastqs = ''

    if len(res_sub['projid']) == 2:
        fastqs = f"{res_sub['Openmind_Destination'][0]},{res_sub['Openmind_Destination'][1]}"
    else:
        fastqs = res_sub['Openmind_Destination'][0]


    out_dir = os.path.join(root, str(num))

    output = f"""#!/bin/bash
#SBATCH -t 47:00:00
#SBATCH -n 32
#SBATCH --mem=300G
#SBATCH --output=/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/Tsai/Preprocessing/Counts/SocIsl/Batch_Scripts_Outs/slurm-%j.out
#SBATCH --error=/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Data/Tsai/Preprocessing/Counts/SocIsl/Batch_Scripts_Err/slurm-%j.err
#SBATCH --mail-user=nkhera@college.harvard.edu
#SBATCH --mail-type=FAIL
export PATH=/om2/user/$USER/apps/yard/cellranger-8.0.0:$PATH
cellranger count --create-bam false --include-introns true --nosecondary --r1-length 26 --id {libID} --transcriptome=/orcd/data/lhtsai/001/om2/mabdel03/yard/references/human/refdata-gex-GRCh38-2020-A --sample {libID} --fastqs {fastqs} --output-dir={out_dir}
"""
    filename = str(num)+'_count.sh' #name the file for the count batch script --> libID_count.sh
    f = open(os.path.join(parent_scripts, filename), 'w') #make the file
    f.write(output) #write to the file
    f.close()
    sbatch_command = f'sbatch {os.path.join(parent_scripts, filename)}' #submit sbatch command
    process = subprocess.Popen(sbatch_command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    
    

In [11]:

    sbatch_command = f'sbatch {os.path.join(parent_scripts, filename)}' #submit sbatch command
    process = subprocess.Popen(sbatch_command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()